In [53]:
import tweepy
import pandas as pd
import os
import sys
import pymongo
import requests
import translators as ts
from langdetect import detect
from json_parsers import *
from sqlalchemy import create_engine
import translators as ts

In [39]:
engine = create_engine('postgresql://postgres@localhost:5432/frenchgp')
data_df.to_sql('zzztest', engine)

In [30]:
MONGO_URI = os.getenv("MONGO_URI")
from pymongo import MongoClient
client_db =  'stream'
client_col = 'frenchgp' #args.collection

# source and target connections
client = MongoClient(MONGO_URI)

# batch size
batch_size = 1000

# database and collection names
db = client[client_db]
col = db[client_col]

# IMPORT

In [110]:
df_in = pd.DataFrame(list(col.find({})))
df_in = df_in.drop(['matching_rules'],1)

# DEFINE FUNCTIONS

In [99]:
def getData(df):
    """
    Convert the nexted json inside the data column to its own dataframe
    """
    df['author_id'] = df.apply(lambda df: parse_json(df, 'data', 'author_id'), 1)
    df['created_at'] = df.apply(lambda df: parse_json(df, 'data', 'created_at'), 1)
    df['geo'] = df.apply(lambda df: parse_json(df, 'data', 'geo'), 1)
    df['tweet_id'] = df.apply(lambda df: parse_json_exact(df, 'data', 'id'), 1)
    df['raw_text'] = df.apply(lambda df: parse_json(df, 'data', 'text'), 1)
    df = df[['tweet_id','author_id','created_at','raw_text']]
    return df


 def getTranslation(df):
    clean_text = df['clean_text']
    translated_text = ts.google(clean_text, if_use_cn_host=True)
    return translated_text


def getCleanText(df):
    clean_text = ''.join(e for e in df['raw_text'] if e.isascii())
    clean_text = ''.join(e for e in clean_text if e not in ["!", "@", "#"])
    return clean_text


def getLanguage(df):
    clean_text = ''.join(e for e in df['raw_text'] if e.isascii())
    clean_text = ''.join(e for e in clean_text if e not in ["!", "@", "#"])
    language = detect(clean_text)
    return language   


def getUsers(df):
    users = df['includes']['users']
    return users


def getUserDataframe(df):
    df['user_created_at'] = df.apply(lambda df: parse_json(df, 'users', 'created_at'), 1)
    df['user_id'] = df.apply(lambda df: parse_json(df, 'users', 'id'), 1)
    df['location'] = df.apply(lambda df: parse_json(df, 'users', 'location'), 1)
    df['name'] = df.apply(lambda df: parse_json_exact(df, 'users', 'name'), 1)
    df['username'] = df.apply(lambda df: parse_json(df, 'users', 'username'), 1)
    df = df.drop(['users'],1)
    return df

,tweet_id,author_id,created_at,raw_text
0,1405185477834051584,1364879051438002176,2021-06-16T15:28:19.000Z,Esteban Ocon signs new Alpine F1 deal until 20...
1,1405185576085733380,1350309702,2021-06-16T15:28:43.000Z,BIG UP TO ESTEBAN OCON WHOSE SIGNED A NEW 3 YE...
2,1405185596771995653,1368546012067995650,2021-06-16T15:28:48.000Z,Neuer Dreijahresvertrag: Esteban Ocon verlänge...
3,1405185683753422851,4843529163,2021-06-16T15:29:08.000Z,"#Formula1 , Esteban #Ocon rinnova con #Alpine ..."
4,1405185708323741697,493042610,2021-06-16T15:29:14.000Z,"#F1: ufficiale, #Ocon pilota #Alpine fino al 2..."
5,1405185726921252866,831236353585246210,2021-06-16T15:29:19.000Z,Alpine heeft de line-up voor volgend jaar rond...
6,1405185742696075265,1357714434265915393,2021-06-16T15:29:22.000Z,French driver signs contract extension with a ...
7,1405185757149605888,1129878879143911426,2021-06-16T15:29:26.000Z,OFICIAL: Esteban Ocon seguirá con @AlpineF1Tea...
8,1405185763189301248,1318271660307472385,2021-06-16T15:29:27.000Z,Esteban Ocon has signed a 3-year deal with Alp...
9,1405185780444712960,3308385844,2021-06-16T15:29:31.000Z,Will Esteban Ocon ever be good enough to fight...


# CLEAN TWEET DATA / TRANSLATE

In [111]:
%%time
data_df = getData(df_in)
data_df['clean_text'] = data_df.apply(lambda data_df: getCleanText(data_df), 1)
data_df['language'] = data_df.apply(lambda data_df: getLanguage(data_df), 1)
data_df_translate = data_df[data_df['language'] != 'en'][['tweet_id','clean_text']]
data_df_translate['translated_text'] = data_df_translate.apply(lambda data_df_translate: getTranslation(data_df_translate), 1)
data_df = data_df \
            .merge(data_df_translate, on='tweet_id', how='left') \
            .rename(index=str, columns={'clean_text_x': 'clean_text'}) \
            .drop('clean_text_y', 1)
data_df['translated_text'] = data_df['translated_text'].fillna(data_df['clean_text'])

/Users/iiyakhruschev/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/iiyakhruschev/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
CPU times: user 239 ms, sys: 16 ms, total: 255 ms
Wall time: 5.01 s


In [112]:
data_df

,tweet_id,author_id,created_at,raw_text,clean_text,language,translated_text
0,1405185477834051584,1364879051438002176,2021-06-16T15:28:19.000Z,Esteban Ocon signs new Alpine F1 deal until 20...,Esteban Ocon signs new Alpine F1 deal until202...,it,Esteban Ocon signs new Alpine F1 deal until202...
1,1405185576085733380,1350309702,2021-06-16T15:28:43.000Z,BIG UP TO ESTEBAN OCON WHOSE SIGNED A NEW 3 YE...,BIG UP TO ESTEBAN OCON WHOSE SIGNED A NEW 3 YE...,en,BIG UP TO ESTEBAN OCON WHOSE SIGNED A NEW 3 YE...
2,1405185596771995653,1368546012067995650,2021-06-16T15:28:48.000Z,Neuer Dreijahresvertrag: Esteban Ocon verlänge...,Neuer Dreijahresvertrag: Esteban Ocon verlnger...,de,New three-year contract: Esteban Ocon renewed ...
3,1405185683753422851,4843529163,2021-06-16T15:29:08.000Z,"#Formula1 , Esteban #Ocon rinnova con #Alpine ...","Formula1 , Esteban Ocon rinnova con Alpine fin...",it,"Formula1, Esteban Ocon renews with Alpine up t..."
4,1405185708323741697,493042610,2021-06-16T15:29:14.000Z,"#F1: ufficiale, #Ocon pilota #Alpine fino al 2...","F1: ufficiale, Ocon pilota Alpine fino al 2024...",it,"F1: official, Ocon Alpine pilot up to 2024 htt..."
5,1405185726921252866,831236353585246210,2021-06-16T15:29:19.000Z,Alpine heeft de line-up voor volgend jaar rond...,Alpine heeft de line-up voor volgend jaar rond...,nl,Alpine has the line-up for next year around.Al...
6,1405185742696075265,1357714434265915393,2021-06-16T15:29:22.000Z,French driver signs contract extension with a ...,French driver signs contract extension with a ...,en,French driver signs contract extension with a ...
7,1405185757149605888,1129878879143911426,2021-06-16T15:29:26.000Z,OFICIAL: Esteban Ocon seguirá con @AlpineF1Tea...,OFICIAL: Esteban Ocon seguir con AlpineF1Team ...,pt,Official: Esteban Ocon Follow Alpinef1team unt...
8,1405185763189301248,1318271660307472385,2021-06-16T15:29:27.000Z,Esteban Ocon has signed a 3-year deal with Alp...,Esteban Ocon has signed a 3-year deal with Alp...,en,Esteban Ocon has signed a 3-year deal with Alp...
9,1405185780444712960,3308385844,2021-06-16T15:29:31.000Z,Will Esteban Ocon ever be good enough to fight...,Will Esteban Ocon ever be good enough to fight...,en,Will Esteban Ocon ever be good enough to fight...


In [33]:


df_in['users'] = df_in.apply(lambda df_in: getUsers(df_in), 1)

In [48]:
users_exploded = df_in[['tweet_id','users']].explode('users')



users_df = getUserDataframe(users_exploded)

In [49]:
users_df

,tweet_id,created_at,user_id,location,name,username
0,1405185477834051584,2021-02-25T10:05:11.000Z,1364879051438002176,India,Khel ख़बर,Khel17934254
1,1405185576085733380,2013-04-13T21:55:37.000Z,1350309702,"Stoke Ash, England",Steven Featley,SFeatley
2,1405185596771995653,2021-03-07T12:56:52.000Z,1368546012067995650,"Unterföhring, Deutschland",Sky Sport Formel 1,skysportformel1
3,1405185683753422851,2016-01-24T22:43:30.000Z,4843529163,Italia,Sportface,sportface2016
4,1405185708323741697,2012-02-15T11:38:09.000Z,493042610,[],FormulaPassion.it,FormulaPassion
5,1405185726921252866,2017-02-13T20:19:25.000Z,831236353585246210,[],Ronald Vording,RonaldVording
6,1405185742696075265,2021-02-05T15:35:34.000Z,1357714434265915393,[],F1 Ramble,F1Ramble
7,1405185757149605888,2019-05-18T22:38:11.000Z,1129878879143911426,"Bogotá, D.C., Colombia",Valentina Peña Orozco,valentinapena98
7,1405185757149605888,2009-04-02T07:08:31.000Z,28297965,[],Alpine F1 Team,AlpineF1Team
8,1405185763189301248,2020-10-19T19:23:56.000Z,1318271660307472385,Worldwide,ASN Motorsports,AsnMotorsports


In [29]:
translator = Translator()
for i in range(len(df)):
    original_text = df['data'][i]['text']
    clean_text = ''.join(e for e in original_text if e.isascii())
    clean_text = ''.join(e for e in clean_text if e not in ["!", "@", "#"])
    
    print(i, '\n original: ', original_text)
    print('clean: ', clean_text)
    print('original language: ', detect(clean_text))
    try:
        print('translation: ',ts.google(clean_text, if_use_cn_host=True))
    except Exception:
        pass

0 
 original:  Esteban Ocon signs new Alpine F1 deal until 2024 https://t.co/zBA9Wr0YTt
clean:  Esteban Ocon signs new Alpine F1 deal until2024 https://t.co/zBA9Wr0YTt
original language:  ca
translation:  Esteban Ocon signs new Alpine F1 deal until2024 https://t.co/zBA9Wr0YTt
1 
 original:  BIG UP TO ESTEBAN OCON WHOSE SIGNED A NEW 3 YEAR DEAL FOR ALPINE F1
WHERE HE'S BEEN SINCE THE START OF 2020
This Following his debut at 2016 Belgian GP, 2 yrs at Force India, (Aston Martin), in 17 &amp; 18 with a year out in 19 as Mercedes 3rd Driver 
has he earned that New contract?
clean:  BIG UP TO ESTEBAN OCON WHOSE SIGNED A NEW 3 YEAR DEAL FOR ALPINE F1
WHERE HE'S BEEN SINCE THE START OF 2020
This Following his debut at 2016 Belgian GP, 2 yrs at Force India, (Aston Martin), in 17 &amp; 18 with a year out in 19 as Mercedes 3rd Driver 
has he earned that New contract?
original language:  en
translation:  BIG UP TO ESTEBAN OCON WHOSE SIGNED A NEW 3 YEAR DEAL FOR ALPINE F1
WHERE HE'S BEEN SINCE THE

In [ ]:
def cleanText(df):
    

In [13]:
import translators as ts

wyw_text = df['data'][4]['text']
# chs_text = '季姬感到寂寞，罗集了一些鸡来养，鸡是那种出自荆棘丛中的野鸡。野鸡饿了唧唧叫，季姬就拿竹箕中的谷物喂鸡。'

# input languages
# print(ts.deepl(wyw_text)) # default: from_language='auto', to_language='en'

## output language_map
# print(ts._deepl.language_map)

# professional field
# print(ts.alibaba(wyw_text, professional_field='general')) # ("general","message","offer")
# print(ts.baidu(wyw_text, professional_field='common')) # ('common','medicine','electronics','mechanics')

# # property
# # rs = [ts.tencent(x) for x in [wyw_text, chs_text]]
# print(ts._tencent.query_count)
# print(dir(ts._tencent))

# # requests
# print(ts.youdao(wyw_text, sleep_seconds=5, proxies={}))

# host service
print(ts.google(wyw_text, if_use_cn_host=True))

# F1: official, #Con pilot #alpine up to 2024 https://t.co/GftagCMhoa


In [23]:
df

,_id,data,includes,matching_rules
0,60ca189ef88ecb49d22d0ab3,"{'author_id': '1364879051438002176', 'created_...",{'users': [{'created_at': '2021-02-25T10:05:11...,"[{'id': 1405185482108157953, 'tag': ''}]"
1,60ca18b5f88ecb49d22d0ab4,"{'author_id': '1350309702', 'created_at': '202...",{'users': [{'created_at': '2013-04-13T21:55:37...,"[{'id': 1405185482108157953, 'tag': ''}]"
2,60ca18baf88ecb49d22d0ab5,"{'author_id': '1368546012067995650', 'created_...",{'users': [{'created_at': '2021-03-07T12:56:52...,"[{'id': 1405185482108157953, 'tag': ''}]"
3,60ca18cff88ecb49d22d0ab6,"{'author_id': '4843529163', 'created_at': '202...",{'users': [{'created_at': '2016-01-24T22:43:30...,"[{'id': 1405185482108157953, 'tag': ''}]"
4,60ca18d5f88ecb49d22d0ab7,"{'author_id': '493042610', 'created_at': '2021...",{'users': [{'created_at': '2012-02-15T11:38:09...,"[{'id': 1405185482108157953, 'tag': ''}]"
5,60ca18d9f88ecb49d22d0ab8,"{'author_id': '831236353585246210', 'created_a...",{'users': [{'created_at': '2017-02-13T20:19:25...,"[{'id': 1405185482108157953, 'tag': ''}]"
6,60ca18ddf88ecb49d22d0ab9,"{'author_id': '1357714434265915393', 'created_...",{'users': [{'created_at': '2021-02-05T15:35:34...,"[{'id': 1405185482108157953, 'tag': ''}]"
7,60ca18e1f88ecb49d22d0aba,"{'author_id': '1129878879143911426', 'created_...",{'users': [{'created_at': '2019-05-18T22:38:11...,"[{'id': 1405185482108157953, 'tag': ''}]"
8,60ca18e2f88ecb49d22d0abb,"{'author_id': '1318271660307472385', 'created_...",{'users': [{'created_at': '2020-10-19T19:23:56...,"[{'id': 1405185482108157953, 'tag': ''}]"
9,60ca18e6f88ecb49d22d0abc,"{'author_id': '3308385844', 'created_at': '202...",{'users': [{'created_at': '2015-06-04T16:01:06...,"[{'id': 1405185482108157953, 'tag': ''}]"
